In [1]:
import os
import datetime
import torch
import random
import pickle
import numpy as np
import pandas as pd

#os.chdir(os.path.join(os.getcwd(), 'LAS Model'))
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from data import SpeechDataset, AudioDataLoader
from listener import Listener
from attend_and_spell import AttendAndSpell
from seq2seq import Seq2Seq
from utils import  train

### Load Training data

In [2]:
def preprocess(s):
    s = s.lower().replace('\n', '')
    return s.translate(str.maketrans('', '', string.punctuation)) # remove punctuation

# Used when each sentence is in a separate text file
def make_train_df(root_dir, dataset, file_ext, csv_file):
    dataset_dir = os.path.join(root_dir, dataset)
    data = []
    files = os.listdir(dataset_dir)
    for f in files:
        if '.txt' in f:
            with open(os.path.join(dataset_dir, f), 'r') as text_file:
                data_list = text_file.readlines()
            for example in data_list:
                path = os.path.join(dataset, str(example.split(' ')[0])) + file_ext   
                sent = preprocess(str(' '.join(example.split(' ')[1:])))
                data.append((path, sent))

    data_df = pd.DataFrame(data, columns=['path', 'sent'])
    data_df.to_csv(os.path.join(root_dir, csv_file), header=None)
    return data_df


root_dir = '../../../Dataset/LibriSpeech'
DEVICE = torch.device('cuda:1') if torch.cuda.is_available() else 'cpu'
print('DEVICE :', DEVICE)

DEVICE : cuda:1


### Load Training data

In [3]:
train_100 = pd.read_csv(os.path.join(root_dir, 'train_100.csv'), header=None, names=['path', 'sent'])
train_360 = pd.read_csv(os.path.join(root_dir, 'train_360.csv'), header=None, names=['path', 'sent'])
train_500 = pd.read_csv(os.path.join(root_dir, 'train_500.csv'), header=None, names=['path', 'sent'])

# combine all of them
train_df = pd.concat([train_100, train_360, train_500])
print("Number of training examples:", train_df.shape[0])
train_df.head()

Number of training examples: 281241


,path,sent
0,dataset_100/103-1240-0000.flac,chapter one missus rachel lynde is surprised m...
1,dataset_100/103-1240-0001.flac,that had its source away back in the woods of ...
2,dataset_100/103-1240-0002.flac,for not even a brook could run past missus rac...
3,dataset_100/103-1240-0003.flac,and that if she noticed anything odd or out of...
4,dataset_100/103-1240-0004.flac,but missus rachel lynde was one of those capab...


In [4]:
# Removing very large sentences
def remove_long_sent(train_df, max_len):
    data = []
    for idx in range(train_df.shape[0]):
        path, sent = train_df.iloc[idx]
        if len(sent) > max_len:
            continue
        data.append((path, sent))
    return pd.DataFrame(data, columns=['path', 'sent'])

max_len = 200
train_df = remove_long_sent(train_df, max_len)
print("Number of training examples:",  train_df.shape[0])

Number of training examples: 165807


### DataLoaders and hyperparams

In [5]:
def get_chars(include_digits=True):
    if include_digits:
        chars = ['<sos>', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', \
                 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', \
                'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', \
                 'x', 'y', 'z', ' ', "'", '<eos>', '<pad>', '<unk>']
    else:
        chars = ['<sos>', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', \
                'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', \
                'y', 'z', ' ', "'", '<eos>', '<pad>', '<unk>']
    print('Number of chars', len(chars))
    return chars


chars = get_chars(include_digits=False)
char_to_token = {c:i for i,c in enumerate(chars)} 
token_to_char = {i:c for c,i in char_to_token.items()}
sos_token = char_to_token['<sos>']
eos_token = char_to_token['<eos>']
pad_token = char_to_token['<pad>']

Number of chars 32


In [6]:
tensorboard_dir = os.path.join('tb_summary')
train_dataset = SpeechDataset(train_df, root_dir, char_to_token)
train_loader = AudioDataLoader(pad_token, train_dataset, batch_size=96, shuffle=True)

### Instantiate model

In [7]:
load = False

if load:
    saved_file = 'Trained Models/Training_2019-12-25 00:09:23.921978/las_model_6'
    model.load_state_dict(torch.load(saved_file))
    start_epoch = int(saved_file[-1]) + 1
    time = os.listdir(tensorboard_dir)[-1]  # use the last one
else:
    start_epoch = 0
    time = str(datetime.datetime.now())

save_dir = os.path.join('trained_models_librispeech', f'Training_{time}')
try:    
    os.mkdir(save_dir);
except FileExistsError:
    pass

In [8]:
input_size = 128    # num rows in instagram
hidden_dim = 256  # 256*2 nodes in each LSTM
num_layers = 3
dropout = 0.1
layer_norm = False   
encoder = Listener(input_size, hidden_dim, num_layers, dropout=dropout, layer_norm=layer_norm)

hid_sz = 256
embed_dim = 30
vocab_size = len(chars)
decoder = AttendAndSpell(embed_dim, hid_sz, encoder.output_size, vocab_size)

hyperparams = {'input_size':input_size, 'hidden_dim':hidden_dim, 
               'num_layers':num_layers,'dropout':dropout, 
               'layer_norm':layer_norm, 'hid_sz':hid_sz, 
               'embed_dim':embed_dim, 'vocab_size':vocab_size}


# Saving hyperparmas
with open(os.path.join(save_dir, 'info.pickle'), 'wb') as f:
    pickle.dump(hyperparams, f)

criterion = nn.CrossEntropyLoss()
model = Seq2Seq(encoder, decoder, criterion, tf_ratio = 1.0, device=DEVICE).to(DEVICE)
model.train()

Seq2Seq(
  (encoder): Listener(
    (layers): ModuleList(
      (0): piBLSTM(
        (lstm): LSTM(128, 256, batch_first=True, bidirectional=True)
        (dp): Dropout(p=0.1, inplace=False)
      )
      (1): piBLSTM(
        (lstm): LSTM(1024, 256, batch_first=True, bidirectional=True)
        (dp): Dropout(p=0.1, inplace=False)
      )
      (2): piBLSTM(
        (lstm): LSTM(1024, 256, batch_first=True, bidirectional=True)
        (dp): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): AttendAndSpell(
    (embedding): Embedding(32, 30)
    (attention_layer): Attention(
      (linear1): Linear(in_features=1280, out_features=640, bias=True)
      (linear2): Linear(in_features=640, out_features=1, bias=True)
    )
    (pre_lstm_cell): LSTMCell(1054, 256)
    (post_lstm_cell): LSTMCell(1280, 256)
    (mlp): Sequential(
      (0): Linear(in_features=256, out_features=32, bias=True)
      (1): ReLU()
      (2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_runnin

### Training

In [9]:
# model.load_state_dict(torch.load(os.path.join(save_dir, 'las_model_1')))
# model.train()

In [ ]:
# optimizer = optim.ASGD(model.parameters(), lr=0.05)  # lr = 0.2 used in paper
optimizer = optim.Adadelta(model.parameters())
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)
log_interval = 5
print_interval = 20

epochs = 20
load = False

summary_dir = os.path.join(tensorboard_dir, time)
writer = SummaryWriter(summary_dir)


for epoch in range(0, epochs):
    print("\nTeacher forcing ratio:", model.tf_ratio)
    train(model, DEVICE, train_loader, optimizer, epoch, print_interval, writer, log_interval)
    scheduler.step()                                    # Decrease learning rate
    torch.save(model.state_dict(), os.path.join(save_dir, f'las_model_{epoch}'))
    model.tf_ratio = max(model.tf_ratio - 0.05, 0.8)    # Decrease teacher force ratio


Teacher forcing ratio: 1.0
Training, Logging: Mean loss of previous 20 batches 

Train Epoch: 0 [1824/165807 (1%)]	Mean Loss : 7.174016	 time 0:01:04.656842:
Train Epoch: 0 [3744/165807 (2%)]	Mean Loss : 7.120739	 time 0:00:56.408542:
Train Epoch: 0 [5664/165807 (3%)]	Mean Loss : 7.097307	 time 0:00:56.588946:
Train Epoch: 0 [7584/165807 (5%)]	Mean Loss : 7.019993	 time 0:00:56.319480:
Train Epoch: 0 [9504/165807 (6%)]	Mean Loss : 6.948741	 time 0:00:56.259772:
Train Epoch: 0 [11424/165807 (7%)]	Mean Loss : 6.893038	 time 0:00:56.275688:
Train Epoch: 0 [13344/165807 (8%)]	Mean Loss : 6.825622	 time 0:00:55.999560:
Train Epoch: 0 [15264/165807 (9%)]	Mean Loss : 6.740560	 time 0:00:55.949626:
Train Epoch: 0 [17184/165807 (10%)]	Mean Loss : 6.660089	 time 0:00:56.187891:
Train Epoch: 0 [19104/165807 (12%)]	Mean Loss : 6.571464	 time 0:00:55.279843:
Train Epoch: 0 [21024/165807 (13%)]	Mean Loss : 6.486125	 time 0:00:56.711603:
Train Epoch: 0 [22944/165807 (14%)]	Mean Loss : 6.414462	 time

Train Epoch: 1 [32544/165807 (20%)]	Mean Loss : 5.995000	 time 0:00:55.641545:
Train Epoch: 1 [34464/165807 (21%)]	Mean Loss : 5.989332	 time 0:00:55.109312:
Train Epoch: 1 [36384/165807 (22%)]	Mean Loss : 5.997731	 time 0:00:55.562200:
Train Epoch: 1 [38304/165807 (23%)]	Mean Loss : 6.006486	 time 0:00:55.716408:
Train Epoch: 1 [40224/165807 (24%)]	Mean Loss : 5.996071	 time 0:00:56.121093:
Train Epoch: 1 [42144/165807 (25%)]	Mean Loss : 5.993966	 time 0:00:55.976035:
Train Epoch: 1 [44064/165807 (27%)]	Mean Loss : 5.987737	 time 0:00:56.057923:
Train Epoch: 1 [45984/165807 (28%)]	Mean Loss : 6.001422	 time 0:00:55.967177:
Train Epoch: 1 [47904/165807 (29%)]	Mean Loss : 5.993463	 time 0:00:56.045572:
Train Epoch: 1 [49824/165807 (30%)]	Mean Loss : 5.991308	 time 0:00:55.700651:


### TEST

In [ ]:
def decode_pred_sent(out):
    pred_sent = []
    for t in out:
        lol = t.max(dim=1)[1].item()
        pred_sent.append(token_to_char[lol])
    return ''.join(pred_sent)


def decode_true_sent(y):
    sent = []
    for t in y:
        sent.append(token_to_char[t.item()])
    return ''.join(sent)

In [ ]:
num_sent = 10
model.eval()

for _ in range(num_sent):
    
    idx = random.randint(0, train_df.shape[0])
    trial_dataset = SpeechDataset(train_df, root_dir, char_to_token)

    x, y = trial_dataset.__getitem__(idx)
    # plt.imshow(x[0,:,:].detach())

    # Model output
    target = y.unsqueeze(dim=0).to(DEVICE)
    data = x.permute(0, 2, 1).to(DEVICE)
    loss, output = model(data, target)
    print("True sent : ", decode_true_sent(y), end='\n\n')
    print("Pred sent : ", decode_pred_sent(output))
    print("Loss :", loss.item())    
    print("\n")